In [1]:
%matplotlib inline

from Bio.Seq import Seq
from Bio import SeqIO
from Bio.Alphabet import IUPAC

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.ensemble import RandomForestClassifier

import shap

In [2]:
#pd.set_option('display.max_rows', 500)

In [3]:
fasta_file = '../DATA/ZIBRA 2/YFV_LACEN_BAHIA/LACEN-BA_CONSENSUS.fasta'

### different CTs

In [4]:
# Gets the sequences IDs
identifiers = [seq_record.id for seq_record in SeqIO.parse(fasta_file, "fasta")]

In [5]:
# Gets the sequences nucleotides
seqs = np.array([list(str(seq_rec.seq)) for seq_rec in SeqIO.parse(fasta_file, "fasta")])

In [6]:
# Creates columns names based on position, starting from 1, to make it consistent with the 
# sequence analysis, which starts at base number 1.
cols = list(range(1, seqs.shape[1]+1))

In [7]:
# Creates dataframe with data
df = pd.DataFrame(seqs, index=identifiers, columns=cols)
df.head()

,1,2,3,4,5,6,7,8,9,10,...,10230,10231,10232,10233,10234,10235,10236,10237,10238,10239
RJ250_YFV_SAO_MIGUEL_DAS_MATAS_2017.03.17\,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
RJ251_YFV_CORDEIROS_2017.03.10\,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
RJ252_YFV_SAO_FELIPE_2017.03.10\,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
RJ253_YFV_SALVADOR_2017.03.14\,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
RJ256_YFV_ALAGOINHAS_2017.03.14\,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N


In [8]:
samples = pd.read_excel('../DATA/ZIBRA 2/YFV_LACEN_BAHIA/Sequências YFV_LACEN-BA_JG_alvaro.xlsx', index_col='ZIBRA Code')
samples = samples[['Host ML', 'Collection_Date', 'Original_Lab_CT value']]
samples.head()

,Host ML,Collection_Date,Original_Lab_CT value
ZIBRA Code,,,
RJ251,Callithrix,2018-02-15,7.59
RJ250,Callithrix,2018-02-15,31.57
RJ252,Callithrix,2018-02-15,27.88
RJ253,Callithrix,2018-02-15,30.17
RJ256,Callithrix,2018-02-15,29.94


In [9]:
import re
#pattern = "^\d+"
#dic={}

In [10]:
#df['ID'] = ''
#df['species'] = ''
#df['date'] = ''
#df['Ct'] = ''

In [11]:
for index_a, sample_a in samples.iterrows():
    #import ipdb; ipdb.set_trace() # debugging starts here
    print(index_a)
    pattern = '^' + index_a + '_'
    regex = re.compile(pattern)
    for index_b, sample_b in df.iterrows():
        print(index_b)
        if regex.search(index_b):
            print(pattern, index_a, index_b)
            df.loc[index_b,'ID'] = index_a
            df.loc[index_b,'species'] = samples.loc[index_a, 'Host ML']
            df.loc[index_b,'date'] = samples.loc[index_a, 'Collection_Date']
            df.loc[index_b,'Ct'] = samples.loc[index_a, 'Original_Lab_CT value']
            break

RJ251
RJ250_YFV_SAO_MIGUEL_DAS_MATAS_2017.03.17\
RJ251_YFV_CORDEIROS_2017.03.10\
^RJ251_ RJ251 RJ251_YFV_CORDEIROS_2017.03.10\
RJ250
RJ250_YFV_SAO_MIGUEL_DAS_MATAS_2017.03.17\
^RJ250_ RJ250 RJ250_YFV_SAO_MIGUEL_DAS_MATAS_2017.03.17\
RJ252
RJ250_YFV_SAO_MIGUEL_DAS_MATAS_2017.03.17\
RJ251_YFV_CORDEIROS_2017.03.10\
RJ252_YFV_SAO_FELIPE_2017.03.10\
^RJ252_ RJ252 RJ252_YFV_SAO_FELIPE_2017.03.10\
RJ253
RJ250_YFV_SAO_MIGUEL_DAS_MATAS_2017.03.17\
RJ251_YFV_CORDEIROS_2017.03.10\
RJ252_YFV_SAO_FELIPE_2017.03.10\
RJ253_YFV_SALVADOR_2017.03.14\
^RJ253_ RJ253 RJ253_YFV_SALVADOR_2017.03.14\
RJ256
RJ250_YFV_SAO_MIGUEL_DAS_MATAS_2017.03.17\
RJ251_YFV_CORDEIROS_2017.03.10\
RJ252_YFV_SAO_FELIPE_2017.03.10\
RJ253_YFV_SALVADOR_2017.03.14\
RJ256_YFV_ALAGOINHAS_2017.03.14\
^RJ256_ RJ256 RJ256_YFV_ALAGOINHAS_2017.03.14\
RJ257
RJ250_YFV_SAO_MIGUEL_DAS_MATAS_2017.03.17\
RJ251_YFV_CORDEIROS_2017.03.10\
RJ252_YFV_SAO_FELIPE_2017.03.10\
RJ253_YFV_SALVADOR_2017.03.14\
RJ256_YFV_ALAGOINHAS_2017.03.14\
RJ257_YFV_CAT

TypeError: can only concatenate str (not "float") to str

In [13]:
df

,1,2,3,4,5,6,7,8,9,10,...,10234,10235,10236,10237,10238,10239,ID,species,date,Ct
RJ250_YFV_SAO_MIGUEL_DAS_MATAS_2017.03.17\,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,RJ250,Callithrix,2018-02-15,31.57
RJ251_YFV_CORDEIROS_2017.03.10\,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,RJ251,Callithrix,2018-02-15,7.59
RJ252_YFV_SAO_FELIPE_2017.03.10\,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,RJ252,Callithrix,2018-02-15,27.88
RJ253_YFV_SALVADOR_2017.03.14\,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,RJ253,Callithrix,2018-02-15,30.17
RJ256_YFV_ALAGOINHAS_2017.03.14\,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,RJ256,Callithrix,2018-02-15,29.94
RJ257_YFV_CATU_2017.03.15\,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,RJ257,Callithrix,2018-02-15,29.25
RJ258_YFV_PEDRAO_2017.03.15\,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,RJ258,Sem informação,2018-02-15,25.81
RJ259_YFV_SALVADOR_2017.03.15\,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,RJ259,Callithrix,2018-02-15,30.51
RJ260_YFV_SANTA_RITA_DE_CASSIA_2017.03.13\,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,RJ260,Alouatta,2018-02-15,27.43
RJ261_YFV_CAMACARI_2017.03.16\,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,RJ261,Callithrix,2018-02-15,29.39


# _"Class"_ vector
Insert a column named _"class"_ in each of the dataframes, indicating to which class they belong. This is the "y" value for the classification algorithm.

In the present analysis, it is hypothesized that genomes from Africa and Asia did not cause microcephaly, and genomes from Oceania (French Polynesia and Micronesia) and the Americas did cause such neurological diseases. Therefore, we investigate significant differences between these two groups. 

- Africa and Asia are class 0.
- Oceania and Americas are class 1.

In [ ]:
df_americas['class'] = np.ones(len(df_americas))
# df_americas.head()

In [ ]:
df_oceania['class'] = np.ones(len(df_oceania))
#df_oceania.tail()

In [ ]:
df_asia['class'] = np.zeros(len(df_asia))
#df_asia.tail()

In [ ]:
df_africa['class'] = np.zeros(len(df_africa))
#df_africa.tail()

# Full dataset creation

Append all regions together in one `DataFrame`

In [ ]:
df = df_americas.append([df_oceania, df_asia, df_africa])
#df

# One-Hot Encoding

Since genomic data is categorical (ACGT), it must be prepared in a way suitable for ML algorithms, i.e., numerical.

Pandas `pd.get_dummies(df)` method applies "one-hot encoding" to the data, transforming each categorical attribute column (with n categories) into n columns with 0's and 1's. It contains 0's in all positions, except for the one corresponding to the value present in that attribute. See below:

This dataset

|sample|base1|base2|
|------|-----|-----|
|s_1   |A    |T    |
|s_2   |A    |G    |
|s_3   |C    |T    |

Becomes

|sample|base1_A|base1_C|base1_G|base1_T|base2_A|base2_C|base2_G|base2_T|
|------|-------|-------|-------|-------|-------|-------|-------|-------|
|s_1   |1      |0      |0      |0      |0      |0      |0      |1      |
|s_2   |1      |0      |0      |0      |0      |0      |1      |0      |
|s_3   |0      |1      |0      |0      |0      |0      |0      |1      |

In [ ]:
df_ohe = pd.get_dummies(df)

In [ ]:
#df_ohe.describe()

In [ ]:
#df_ohe.head()

### Save data to disk
Save DataFrames for later use. Saves time

In [ ]:
df_ohe.to_pickle('./df_ohe.pkl')

In [ ]:
df_ohe = pd.read_pickle('./df_ohe.pkl')

# Create training and target sets

In [ ]:
X = df_ohe.drop('class', axis=1)
y = df_ohe['class']

# Random Forest Classifier

Based on experience, we will use a random forest with 100 trees (we comparer `oob_score_` values for different numbers of trees). 

Set `random state = 0` and `oob_score = True` to allow reproducibility and to use "out of bag" samples to compute accuracy.

In [ ]:
#clf = RandomForestClassifier(n_estimators=100, random_state=0, oob_score=True)

In [ ]:
#clf.fit(X, y)

In [ ]:
#clf.oob_score_

In [ ]:
#predictions = clf.predict(X)
#predictions == Y.values

### Save classifier
Save trained classifier to disk to avoid re-training every time the notebook is run.

In [ ]:
from sklearn.externals import joblib
#joblib.dump(clf, 'saved_random_forest.joblib');

In [ ]:
clf = joblib.load('saved_random_forest.joblib')
print("oob score: ", clf.oob_score_)
clf

# Feature importances

Below is a list containing the feature importances as obtained by the random forest, using SciKit Learn's built-in feature importance measurement. This analyses the impurity decrease for every node of every tree, and computes each attribute's contribution in the forest. An explanation can be found #here [link to page explaining](). 

However, as pointed out in [this study](), this approach to variable importance might not be the most adequate. It can be inconsistent and/or misleading in some cases.

Therefore, we will later use another approach to variable importance (see SHAP below)

In [ ]:
imp = clf.feature_importances_
attr = X.columns

In [ ]:
ohe_importances = pd.Series(imp, index=attr)
#ohe_importances.head()
ohe_importances.shape

# From one-hot encoding to original features' names.
Since "ohe" creates additional columns to accomodate the attributes' categories, we have to go back to the original attribute's names in order to clearly analyse the results, especially where it concerns feature importances.

In [ ]:
import re
pattern = "^\d+"
dic={}

In [ ]:
# The code below sums the importances for each category in each attribute.
for pos in ohe_importances.index:
    attr = re.match(pattern, pos).group()
    if attr not in dic.keys():
        dic[attr] = ohe_importances[pos]
    else:
        dic[attr] += ohe_importances[pos]
    

In [ ]:
original_importances = pd.Series(dic)
original_importances_sorted = original_importances.sort_values(ascending=False)
original_importances_sorted

In [ ]:
max_imp = original_importances.max()
original_index = original_importances.index
loc = pd.Index(original_importances).get_loc(max_imp)
max_pos = original_index[loc]

In [ ]:
imp_threshold = 0.01

In [ ]:
major_importances = original_importances[original_importances > imp_threshold]

In [ ]:
major_importances.sum()

In [ ]:
print('The position with highest importance is nucleotide n{0}, with {1}'.format(max_pos, max_imp))

In [ ]:
locs = np.where(original_importances > imp_threshold)
positions = original_index[locs]
print(positions)

In [ ]:
len(positions)

In [ ]:
plt.scatter(positions, original_importances[positions]);

# Location on the genome
The positions above are the ones that have the largest influence on the classification results.
Together, they account for {{"%.2f" % (100 * major_importances.sum()) + "\%"}} of the prediction capacity. 

However, when we look at where they are located in the genome, most of them are in the 3rd position in the codon, i.e., these SNPs (single nucleotide polymorphisms) do not change the coded aminoacids. Even the few ones that appear on the 1st position in the codon, when inspected, didn't cause the aminoacid to change.

This deserves our attention and discussion. Maybe we are seeing adaptations in "codon usage".

The array below shows where in the codon each SNP occur:

- 0: 3rd position
- 2: 2nd position
- 1: 1st position

In [ ]:
important = original_importances_sorted[original_importances_sorted > imp_threshold]
important_positions = important.index
nums = important_positions.values.astype('int')
resto = nums%3
df_codons = pd.Series(resto, index=important_positions)
df_codons

# SHAP importance

In [ ]:
# load JS visualization code to notebook
shap.initjs()

# explain the model's predictions using SHAP values
# (same syntax works for LightGBM, CatBoost, and scikit-learn models)
explainer = shap.TreeExplainer(clf)
shap_values = explainer.shap_values(X)

# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
#shap.force_plot(explainer.expected_value, shap_values[0,:], X.iloc[0,:])

In [ ]:
array_shap_values = np.array(shap_values)
array_shap_values.shape

In [ ]:
explainer.expected_value

In [ ]:
# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
shap.force_plot(explainer.expected_value[1], array_shap_values[1][0,:], X.iloc[0,:])

# Combine one-hot encoding back to original attributes

In [ ]:
print('shap_values is a table with {0} samples and {1} attributes'.format(shap_values[1].shape[0], shap_values[1].shape[1]))
cols = X.columns
print('X dataset has {0} columns (i.e. attributes)'.format(cols.shape[0]))

In [ ]:
df_shap_values = pd.DataFrame(shap_values[1], index=X.index, columns=X.columns)
print('df_shap_values is a DataFrame with {0} samples and {1} columns'.format(df_shap_values.shape[0], df_shap_values.shape[1]))

In [ ]:
list_shap_original = []

In [ ]:
import re
pattern = "^\d+"
dic={}

In [ ]:
for i, sample in df_shap_values.iterrows():
    dic = {}
    # The code below sums the importances for each category in each attribute.
    for pos in sample.index:
        attr = re.match(pattern, pos).group()
        if attr not in dic.keys():
            dic[attr] = sample[pos]
        else:
            dic[attr] += sample[pos]
    df_sample = pd.DataFrame(dic, index=[i])
    list_shap_original.append(df_sample)


In [ ]:
shap_original = pd.concat(list_shap_original, axis=0)

In [ ]:
shap_original.shape

In [ ]:
data = df.drop('class', axis=1).values
data.shape
X_original = pd.DataFrame(data, index=shap_original.index, columns=shap_original.columns)
X_original.head()

Now the dataframe `shap_original` contains the shap values for the correct attributes. 

As suggested by SHAP's developer [here](https://github.com/slundberg/shap/issues/397), the values were summed for all categories in each attribute, for each sample.

# Sort attributes by importance
## Select the 30 most important ones

In [ ]:
# Since shap values are positive or negative depending on which direction they push the result, I will take the abs value to determine the most important ones.
shap_abs = abs(shap_original)

In [ ]:
# Here, I'll summ all abs shap values in each column.
# This way, I can have an idea of which columns have the largest overall effect.
# Then I'll select the 30 largest.
shap_sum = shap_abs.sum(axis=0)


In [ ]:
max_imp = shap_sum.max()
original_index = shap_sum.index
loc = pd.Index(shap_sum).get_loc(max_imp)
max_pos = original_index[loc]

### Select 30 most important

In [ ]:
sorted_shap_sum = shap_sum.sort_values(ascending=False)
sorted_shap_sum.head()

In [ ]:
best_shap = sorted_shap_sum[0:30]
best_shap.head()

In [ ]:
cols = list(best_shap.index)

In [ ]:
X_best_attrib = X_original.loc[:, cols]
X_best_attrib.head()

In [ ]:
best_shap_values = shap_original[cols]
best_shap_values.iloc[0,:].values

In [ ]:
X_best_attrib.iloc[0,:]

In [ ]:
# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
shap.force_plot(explainer.expected_value[1], best_shap_values.iloc[340,:].values, X_best_attrib.iloc[340,:])

# !!!Corrigir esse nome
para a funcao shap.force_plot, o shap values deve ser um nd.numpy.array, ou seja, os valores somente.

A variavel `best_shap_values` é um dataframe, preciso de um np.array

In [ ]:
best_shap_values.head()
best_shap_values_values = best_shap_values.values

In [ ]:
# visualize the training set predictions
shap.force_plot(explainer.expected_value[1], best_shap_values_values, X_best_attrib)

In [ ]:
plt.scatter(np.linspace(1, 341, 341), best_shap_values.iloc[:,0]);

# stopped here

In [ ]:
expect = explainer.expected_value[1]
expect

In [ ]:
sp_v = shap_values[1][0,0:5]
sp_v

In [ ]:
X_reduced = X.iloc[0,0:5]
X_reduced

In [ ]:
# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
shap.force_plot(expect, sp_v, X_reduced)

In [ ]:
imp = shap_values[0][0]
col = X.columns
df_imp = pd.Series(imp, index=col)
df_imp = df_imp.sort_values(ascending=False)
df_imp

In [ ]:
# The code below sums the importances for each category in each attribute.
for pos in df_imp.index:
    attr = re.match(pattern, pos).group()
    if attr not in dic.keys():
        dic[attr] = df_imp[pos]
    else:
        dic[attr] += df_imp[pos]
    

In [ ]:
original_shap = pd.Series(dic)
original_shap_values = original_shap.values
original_X = X.loc[list(original_shap.index)]

In [ ]:
original_shap

# Treat importance values from one-hot encoding to original attributes

[Questions about SHAP handling categorical variables #397](https://github.com/slundberg/shap/issues/397)

Hey!

1 - You should use SUM (assuming you don't want to break it out by category). Because that will measure the total effect of all the categories, and so capture the impact of that feature before one-hot encoding.

2 - I think sum is already a very well motivated way of computing the Shapley values for the group. The nice thing about Shapley values additivity is that it makes sense to let the credit of a group be the sum of the credit assigned to each member. A less-obvious feature is also to tell KernelExplainer to treat a whole group of features as a single entity by using the shap.common.DenseData object (which also makes the method faster). The only difference between summing up the credit of each feature and treating them all as a single "group feature" is that when credit for interaction effects is split evenly among the set of all participating features, the size of that set could change if you collapse some features to a single group.